**1. 데이터 다운**  
https://www.kaggle.com/bittlingmayer/amazonreviews

**2. 패키지 선언**

In [0]:
import os
import numpy as np
import nltk
import random
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

**3. 학습 데이터와 평가 데이터**



In [0]:
def read_dataset(dataset_type):
  max_seq_len = 0
  with open("/content/%s.txt" % dataset_type, "r", encoding="utf-8") as fr_handle:
    labels, sentences, tokenized_sentences = [], [], []
    for line in fr_handle:
      if line.strip() == 0:
        continue
      label = line.split(' ')[0]
      label = 0 if label == "__label__1" else 1 # 부정이면 0, 긍정이면 1

      sentence = ' '.join(line.split(' ')[1:])
      tokenized_sentence = nltk.word_tokenize(sentence)
      max_seq_len = max(max_seq_len, len(tokenized_sentence))

      labels.append(label)
      sentences.append(sentence)
    
    return labels, sentences, max_seq_len

TRAIN_LABELS, TRAIN_SENTENCES, TRAIN_MAX_SEQ_LEN = read_dataset("train")
TEST_LABELS, TEST_SENTENCES, TEST_MAX_SEQ_LEN = read_dataset("test")
MAX_SEQUENCE_LEN = max(TRAIN_MAX_SEQ_LEN, TEST_MAX_SEQ_LEN)

print("Train : ", len(TRAIN_SENTENCES))
for train_label, train_sent in zip(TRAIN_LABELS, TRAIN_SENTENCES[0:10]):
  print(train_label, ':' ,train_sent)

print()
print("Test : ", len(TEST_SENTENCES))
for test_label, test_sent in zip(TEST_LABELS, TEST_SENTENCES[0:10]):
  print(test_label, ':' ,test_sent)

print("MAX_SEQUENCE_LEN", MAX_SEQUENCE_LEN)
with open("/content/vocab.txt", "r", encoding="utf-8") as vocab_handle:
  VOCAB = [line.strip() for line in vocab_handle if len(line.strip()) > 0]
  
  print("Total vocabulary", VOCAB)

**4. Keras를 통한 전처리 과정**  
1) Text를 tokenize하여 id 값으로 변경해 줍니다. (tokenizer.texts_to_sequences)  
2) id로 변경해준 문장들을 모두 문장 최대 길이로 padding 처리해 줍니다. (pad_sequences)


In [0]:
tokenizer = Tokenizer(num_words=len(VOCAB), lower=True, char_level=False)
tokenizer.fit_on_texts(TRAIN_SENTENCES)
TRAIN_SEQUENCES = tokenizer.texts_to_sequences(TRAIN_SENTENCES)
TEST_SEQUENCES = tokenizer.texts_to_sequences(TEST_SENTENCES)
VOCAB_SIZE = len(tokenizer.word_index) + 1

print(TRAIN_SENTENCES[2])
print(TRAIN_SEQUENCES[2])

X_train = pad_sequences(TRAIN_SEQUENCES, padding='post', maxlen=MAX_SEQUENCE_LEN)
X_test = pad_sequences(TEST_SEQUENCES, padding='post', maxlen=MAX_SEQUENCE_LEN)
print("PAD_SEQUENCES COMPLETES")
print(X_train[0])
print(MAX_SEQUENCE_LEN)

**5. 모델 설정**  
1) Random으로 초기화된 임베딩이 아닌 pre-trained 된 GLoVE 임베딩으로 학습하고자 합니다.  
2) 따라서 학습 코퍼스에 있는 단어들 중 GLoVE 임베딩에 있는 단어들을 GLoVE 임베딩으로 초기화 해줍니다.  
3) 본 실험에서는 GLoVE 임베딩 크기가 50인 것과 100인 것을 통해 실험을 진행해 봅니다.  

In [0]:
def create_embedding_matrix(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    with open(filepath) as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]

    return embedding_matrix

EMBEDDING_DIM = 50 #100
embedding_matrix = create_embedding_matrix(
    '/content/glove.6B.50d.txt',
    tokenizer.word_index, EMBEDDING_DIM
    )

**6. Accuracy와 Loss 시각화**  

In [0]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')

def plot_history(history):
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training acc')
    plt.plot(x, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

**7. CNN 모델 선언**  
1) Convolution 필터를 앞서 배운 것처럼 여러 개를 사용해 봅시다.  
2) 본 모델은 필터의 Window Size가 2, 3, 4, 5인 필터 각 100개씩을 사용해 모델을 학습합니다. 다양한 크기의 필터를 사용하면 성능이 더 올라갈까요?  
3) 정답은 "올라갑니다." 입니다. Convolution 필터가 보는 단어의 갯수가 다양하게 되기 때문에 문장의 local 정보와 global 정보 모두를 학습할 수 있게 됩니다. 그럼 실험을 통해 확인해 볼까요?  

In [0]:
from keras.models import Sequential
from keras import layers
from keras.models import Model

seq_input = layers.Input(shape=(MAX_SEQUENCE_LEN,), dtype='int32')
seq_embedded = layers.Embedding(VOCAB_SIZE, 
                           EMBEDDING_DIM, 
                           weights=[embedding_matrix], 
                           input_length=MAX_SEQUENCE_LEN, 
                           trainable=True)(seq_input)

filters = [2,3,4,5]
conv_models = []
for filter in filters:
  conv_feat = layers.Conv1D(filters=100, 
                            kernel_size=filter, 
                            activation='relu',
                            padding='valid')(seq_embedded)
  pooled_feat = layers.GlobalMaxPooling1D()(conv_feat)
  conv_models.append(pooled_feat)

conv_merged = layers.concatenate(conv_models, axis=1)

model_output = layers.Dropout(0.2)(conv_merged)
model_output = layers.Dense(10, activation='relu')(model_output)
logits = layers.Dense(1, activation='sigmoid')(model_output)

model = Model(seq_input, logits)
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

#학습 시작
history = model.fit(X_train, TRAIN_LABELS,
                    epochs=10,
                    verbose=True,
                    validation_data=(X_test, TEST_LABELS),
                    batch_size=128)
# 결과 시각화
plot_history(history)